# Loads turtle files and stores them into the graph

In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Globals and Imports

In [2]:
from os.path import isfile, join
from os import listdir
from pathlib import Path
import sys
from rdflib import RDF, SDO, Graph, URIRef, BNode, Literal
import logging
import json
import requests
from urllib import parse
from rdflib import RDFS

In [3]:
# for loading the captions

base = "D:\\Scraped\\youtube\\rdf\\captions"
host = "https://entitygraph.azurewebsites.net"
path = "api/entities"
key = "Tzre7295T10z1K"

## Functions

In [4]:
from urllib.parse import quote, quote_plus


def loadFiles(): 
   return [f for f in listdir(base) if isfile(join(base, f))]

def openFile(filename: str) -> str: 
    file: Path =  Path.joinpath(Path(base), filename)
    content = file.open(mode='r', encoding="utf-8").read()
    return content

def loadGraph(content: str) -> Graph:   
    g = Graph()
    g.parse(format="n3", data=content)
    return g

def extractIdentifier(node: URIRef) -> str: 
    uri = str(node); 
    return uri[uri.rindex("/")+1:]

def getEntity(id: str, property: str) -> Graph: 
    g = Graph()
    try: 
        encodedId = quote(id)
        print("GET: "+f"{host}/{path}/{encodedId}?property={property}")
        response = requests.get(
            url=f"{host}/{path}/{encodedId}?property={property}", 
            headers= {
                "X-API-KEY": key, 
                "Accept": "text/n3"
            })
        
        if not response.ok: 
            print("Request to read entity failed with message: "+response.reason)
            return g
        
        g.parse(format="n3", data=response.content)
            
    
    except Exception as err:
        print("failed to read entity. reason: "+str(err))            
    finally: 
        return g


def storeValue(value: str, prefixedKey: str, entityId: str) -> bool: 
    try: 
        print("POST: "+f"{host}/{path}/{entityId}/values/{prefixedKey}")
        response = requests.post(
            url=f"{host}/{path}/{entityId}/values/{prefixedKey}", 
            data=value.encode("utf-8"), 
            headers= {
                "X-API-KEY": key, 
                "Content-Type": "text/plain", 
                "Accept": "text/turtle"
            })
        
        if not response.ok: 
            print("Request to store updated value failed with message: "+response.reason)
    except Exception as err:
        print("failed to store. reason: "+str(err))
    


## Script

In [71]:


# script testing for one source file only
file = loadFiles()[0]
print(file)     
data: str = openFile(filename=file)
g: Graph  = loadGraph(data)

for s, p, o in g.triples((None, RDF.type, SDO.VideoObject)):
    extId: str = g.value(s, SDO.identifier)
    entity: Graph = getEntity(extId, "sdo.identifier")
    for node in entity.subjects(RDF.type, SDO.LearningResource): 
        entityId = extractIdentifier(node)
        for obj in g.objects(None, SDO.caption): 
            storeValue(value=str(obj), prefixedKey="sdo.caption", entityId=entityId)
    
    
 

captions_--SZOi73UGU.ttl
GET: http://localhost:8080/api/entities/--SZOi73UGU?property=sdo.identifier
POST: http://localhost:8080/api/entities/mbeusafpxxfe/values/sdo.caption
POST: http://localhost:8080/api/entities/mbeusafpxxfe/values/sdo.caption


In [5]:


# script running through all files
for file in loadFiles():
    data: str = openFile(filename=file)
    g: Graph  = loadGraph(data)

    for s, p, o in g.triples((None, RDF.type, SDO.VideoObject)):
        extId: str = g.value(s, SDO.identifier)
        entity: Graph = getEntity(extId, "sdo.identifier")
        for node in entity.subjects(RDF.type, SDO.LearningResource): 
            entityId = extractIdentifier(node)
            for obj in g.objects(None, SDO.caption): 
                storeValue(value=str(obj), prefixedKey="sdo.caption", entityId=entityId)
    
    
 

GET: https://entitygraph.azurewebsites.net///api/entities/--SZOi73UGU?property=sdo.identifier
POST: https://entitygraph.azurewebsites.net///api/entities/6nwwdy8ft90a/values/sdo.caption
POST: https://entitygraph.azurewebsites.net///api/entities/6nwwdy8ft90a/values/sdo.caption
GET: https://entitygraph.azurewebsites.net///api/entities/--V9j3gJF_A?property=sdo.identifier
POST: https://entitygraph.azurewebsites.net///api/entities/koja2covxtmr/values/sdo.caption
POST: https://entitygraph.azurewebsites.net///api/entities/koja2covxtmr/values/sdo.caption
GET: https://entitygraph.azurewebsites.net///api/entities/-0ENvOK1Rfg?property=sdo.identifier
POST: https://entitygraph.azurewebsites.net///api/entities/of3hoqtkwn9h/values/sdo.caption
POST: https://entitygraph.azurewebsites.net///api/entities/of3hoqtkwn9h/values/sdo.caption
GET: https://entitygraph.azurewebsites.net///api/entities/-1ecuhQJwNU?property=sdo.identifier
POST: https://entitygraph.azurewebsites.net///api/entities/f0aciitdflmm/values